# Imports  

In [ ]:
from gliderad2cp import process_currents, process_shear, process_bias, tools, download_example_data
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

There are several options that can be customised including correlation thresholds, velocity regridding options, and offsets to correct for transducer misalignment. There are all set in the `options` dict.

In [ ]:
options = tools.get_options(xaxis=1, yaxis=3, QC_correlation_threshold=80,
                            QC_amplitude_threshold=80, QC_velocity_threshold=1.5, 
                            velocity_dependent_shear_bias_correction=False, 
                            shear_bias_regression_depth_slice=(10,1000))

# Load data

`gliderad2cp` requires a netCDF file from a Nortek AD2CP which can be produced using the Nortek MIDAS software and a timeseries of glider data. This timeseries can be read from a netCDF, csv, or parquet file, or passed in as an xarray DataSet or pandas Dataframe. The essential variables are:

- "time"
- "temperature"
- "salinity"
- "latitude"
- "pressure"
- "longitude"
- "profile_number"
- "declination"

There are several example datasets available from the function `load_sample_dataset`. We use one of datasets in this notebook

In [ ]:
data_file = download_example_data.load_sample_dataset(dataset_name="sea055_M82.nc")
adcp_file = download_example_data.load_sample_dataset(dataset_name="sea055_M82.ad2cp.00000.nc")

# Step 1: calculate velocity shear

This is handled by the wrapper function `process_shear.process`. The individual steps of processing are detailed in the [documentation](https://www.flow-lab.org/gliderad2cp/).

The output of this function is a gridded xarray dataset including glider relative velocities and profiles of eastward, northward, and vertical velocities SH_E, Sh_N and Sh_U.

In [ ]:
ds_adcp = process_shear.process(adcp_file, data_file, options)

In [ ]:
plt.subplots(figsize=(12, 8))
plt.pcolormesh(ds_adcp.time, ds_adcp.gridded_bin, ds_adcp.Sh_E.T, cmap=cmo.balance)
plt.colorbar(label='s-1')
plt.clim([-0.05, 0.05])
plt.xlim(np.datetime64("2024-10-11"), np.datetime64("2024-10-12"))
plt.ylim(0, 30)
plt.title('Vertical shear of eastward velocity')
plt.gca().invert_yaxis()

# Step 2: shear to velocity

After calculating velocity shear, this can be integrated and referenced to estimate earth relative velocity profiles.

The function `process_currents.process` handles this step, returning DAC referenced velocity profiles.

### Get pre and post dive GPS locations from glider data

To calculate dive average current we require more variables, including estimates of the glider's movement through the water. For this, we need the pre and post dive GPS locations of the glider. This calculation varies between glider models, processing tools and glider firmware versions. See the documentation for more examples of this calculation and make use of the verification plots

In [ ]:
data = xr.open_dataset(data_file)
gps_predive = []
gps_postdive = []

dives = np.round(np.unique(data.dive_num))

_idx = np.arange(len(data.dead_reckoning.values))
dr  = np.sign(np.gradient(data.dead_reckoning.values))

for dn in dives:
    _gd = data.dive_num.values == dn
    if all(np.unique(dr[_gd]) == 0):
        continue

    _post = -dr.copy()
    _post[_post != 1] = np.nan
    _post[~_gd] = np.nan

    _pre = dr.copy()
    _pre[_pre != 1] = np.nan
    _pre[~_gd] = np.nan

    if any(np.isfinite(_post)):
        # The last -1 value is when deadreckoning is set to 0, ie. GPS fix. This is post-dive.
        last  = int(np.nanmax(_idx * _post))
        gps_postdive.append(np.array([data.time[last].values, data.longitude[last].values, data.latitude[last].values]))

    if any(np.isfinite(_pre)):
        # The first +1 value is when deadreckoning is set to 1, the index before that is the last GPS fix. This is pre-dive.
        first = int(np.nanmin(_idx * _pre))-1 # Note the -1 here.
        gps_predive.append(np.array([data.time[first].values, data.longitude[first].values, data.latitude[first].values]))

gps_predive = np.vstack(gps_predive)
gps_postdive = np.vstack(gps_postdive)

We expect `gps_postdive` and `gps_predive` to show as vertical blue and red lines respectively at the beginning and end of a glider surfacing manouvre. In a mission with multiple no-surface dives, as shown in the example below, the dives where the gliders does not surface to fix GPS, do not get assigned `gps_postdive` and `gps_predive`.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(data.time, data.depth, color='k')
for predive in gps_predive:
    ax.axvline(predive[0], color='r')
ax.axvline(predive[0], color='r', label='GPS predive')

for postdive in gps_postdive:
    ax.axvline(postdive[0], color='b')
ax.axvline(postdive[0], color='b', label='GPS postdive')
ax.invert_yaxis()
ax.set(xlim=(np.datetime64("2024-10-11T00:30"), np.datetime64("2024-10-11T03:30")), ylabel='Depth (m)')
ax.legend();

In [ ]:
currents, DAC = process_currents.process(ds_adcp, gps_predive, gps_postdive, options)

### Plot DAC referenced currents

In [ ]:

plt.figure(figsize=(20,6))

plt.subplot(121)
plt.pcolormesh(currents.velocity_E_DAC_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])

plt.subplot(122)
plt.pcolormesh(currents.velocity_N_DAC_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])

# Step 3: Estimate and correct shear bias

This additional, optional processing step attempts to correct for along beam shear bias.

In [ ]:
currents = process_bias.process(currents,options)

### Compare

The following three plots contrast the eastward velocities from integration of shear, after referencing to DAC and following shear bias correction

In [ ]:
plt.figure(figsize=(9,14))
import cmocean.cm as cmo

plt.subplot(311)
plt.pcolormesh(currents.velocity_E_no_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])
plt.title('No referencing')

plt.subplot(312)
plt.pcolormesh(currents.velocity_E_DAC_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])
plt.title('DAC referencing')

plt.subplot(313)
plt.pcolormesh(currents.velocity_E_DAC_reference_sb_corrected, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])
plt.title('DAC referencing, bias correction')
